In [ ]:
# The fuck was i thinking on making this

import time
import cv2
import numpy as np
from collections import deque
from mss import mss
from ultralytics import YOLO
import keyboard
import win32api
import ctypes
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt

# Load YOLO
model = YOLO("best11s.pt")
model.to("cuda")

# DeepSORT init
tracker = DeepSort(
    max_age=2,  # Balanced for retention and speed
    n_init=1,
    nms_max_overlap=1.0,
    embedder="mobilenet",
    half=True,
)

auto_aim = False
target_lock = None

# CONSTANTS
MON = {"top": 0, "left": 0, "width": 1024, "height": 768}
MODEL_INPUT = 416  # Reduced for faster inference. It doesn't do jack shit tbfh
SCREEN_W = ctypes.windll.user32.GetSystemMetrics(0)
SCREEN_H = ctypes.windll.user32.GetSystemMetrics(1)
YOLO_CONF = 0.15 
IOU_SKIP_THRESH = 0.2
SAFE_NEW_IOU = 0.1
FORCE_DROP_IOU = 0.5
MIN_CENTER_DISTANCE = 30.0
MAX_CURSOR_DISTANCE = 800.0
MIN_MOVEMENT_THRESHOLD = 10.0

def letterbox_image(img, new_size=MODEL_INPUT, color=(114,114,114)):
    h, w = img.shape[:2]
    scale = min(new_size / w, new_size / h)
    nw, nh = int(w * scale), int(h * scale)
    img_resized = cv2.resize(img, (nw, nh))
    pad_x = (new_size - nw) // 2
    pad_y = (new_size - nh) // 2
    top, bottom = pad_y, new_size - nh - pad_y
    left, right = pad_x, new_size - nw - pad_x
    img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right,
                                    cv2.BORDER_CONSTANT, value=color)
    return img_padded, scale, pad_x, pad_y

def unletterbox_box(x1, y1, x2, y2, scale, pad_x, pad_y):
    x1_u = (x1 - pad_x) / scale
    y1_u = (y1 - pad_y) / scale
    x2_u = (x2 - pad_x) / scale
    y2_u = (y2 - pad_y) / scale
    return x1_u, y1_u, x2_u, y2_u

def iou_box(a, b):
    x1, y1, x2, y2 = a
    ox1, oy1, ox2, oy2 = b
    inter_x1 = max(x1, ox1)
    inter_y1 = max(y1, oy1)
    inter_x2 = min(x2, ox2)
    inter_y2 = min(y2, oy2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area1 = max(0, x2 - x1) * max(0, y2 - y1)
    area2 = max(0, ox2 - ox1) * max(0, oy2 - oy1)
    union = area1 + area2 - inter_area
    return inter_area / union if union > 0 else 0.0

def clamp_box_to_image(box, img_w, img_h):
    x1, y1, x2, y2 = box
    x1 = max(0, min(img_w - 1, x1))
    x2 = max(0, min(img_w - 1, x2))
    y1 = max(0, min(img_h - 1, y1))
    y2 = max(0, min(img_h - 1, y2))
    if x2 <= x1 or y2 <= y1:
        return None
    return (x1, y1, x2, y2)

def center_distance(box1, box2):
    x1, y1, x2, y2 = box1
    ox1, oy1, ox2, oy2 = box2
    cx1, cy1 = (x1 + x2) / 2.0, (y1 + y2) / 2.0
    cx2, cy2 = (ox1 + ox2) / 2.0, (oy1 + oy2) / 2.0
    return np.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)

def point_distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def is_path_safe(current_pos, target_pos, class1_boxes, margin=20.0, max_distance=MAX_CURSOR_DISTANCE):
    cx, cy = current_pos
    tx, ty = target_pos
    distance = point_distance((cx, cy), (tx, ty))
    if distance > max_distance:
        print(f"Path rejected: Distance {distance:.2f} exceeds max {max_distance}")
        return False
    for x1, y1, x2, y2 in class1_boxes:
        x1_m, y1_m, x2_m, y2_m = x1 - margin, y1 - margin, x2 + margin, y2 + margin
        if x1_m <= tx <= x2_m and y1_m <= ty <= y2_m:
            return False
        dx, dy = tx - cx, ty - cy
        # Fixed 3 steps for path checking
        for t in np.linspace(0, 1, 3):
            px = cx + t * dx
            py = cy + t * dy
            if x1_m <= px <= x2_m and y1_m <= py <= y2_m:
                return False
    return True

try:
    ctypes.windll.user32.SetProcessDPIAware()
except Exception:
    pass

def startAutoSlicer(visualize=True):
    global auto_aim, target_lock

    fps_deque = deque(maxlen=60)
    frame_count = 0
    mouse_pressed = False  # Track mouse button state

    with mss() as sct:
        while True:
            t0 = time.time()
            frame_count += 1

            # Check keyboard input every 5 frames
            if frame_count % 5 == 0 and keyboard.is_pressed("e"):
                auto_aim = not auto_aim
                print(f"Auto Aim {'ON' if auto_aim else 'OFF'}")
                time.sleep(0.005)  # Reduced delay

            img = np.asarray(sct.grab(MON))[:, :, :3]
            img_h, img_w = img.shape[:2]

            img_resized, scale, pad_x, pad_y = letterbox_image(img, new_size=MODEL_INPUT)

            # YOLO inference
            results = model(img_resized, stream=False, conf=YOLO_CONF, iou=0.2, int8=True, verbose=False)

            detections = []
            class0_dets = []
            class1_boxes = []  # Directly store confirmed class 1 boxes
            for box in results[0].boxes:
                cls = int(box.cls[0].cpu())
                if cls not in (0, 1):
                    continue
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(float)
                ux1, uy1, ux2, uy2 = unletterbox_box(x1, y1, x2, y2, scale, pad_x, pad_y)
                clamped = clamp_box_to_image((ux1, uy1, ux2, uy2), img_w, img_h)
                if clamped is None:
                    continue
                ux1, uy1, ux2, uy2 = clamped
                w = ux2 - ux1
                h = uy2 - uy1
                conf = float(box.conf[0].cpu())
                if cls == 0:
                    detections.append(([int(ux1), int(uy1), int(w), int(h)], conf, cls))
                    class0_dets.append((ux1, uy1, ux2, uy2))
                else:
                    class1_boxes.append((ux1, uy1, ux2, uy2))  # Treat class 1 detections as confirmed boxes

            print(f"Class 0 dets: {len(class0_dets)}, Class 1 dets: {len(class1_boxes)}")

            # DeepSort tracking for class 0 only
            tracks = tracker.update_tracks(detections, frame=img)
            class0_tracks = [t for t in tracks if t.det_class == 0 and t.is_confirmed()]
            print(f"Confirmed class 0 tracks: {len(class0_tracks)}")

            target_center = None
            chosen_track = None
            path_to_draw = None
            is_fallback = False

            current_cursor_pos = win32api.GetCursorPos()
            current_cursor_pos = (current_cursor_pos[0] - MON["left"], current_cursor_pos[1] - MON["top"])

            # Single-pass target selection
            best_track = None
            best_distance = float('inf')
            last_track_id = getattr(startAutoSlicer, "last_track_id", None)

            for t in class0_tracks:
                x1, y1, x2, y2 = t.to_ltrb()
                cx, cy = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                cursor_distance = point_distance(current_cursor_pos, (cx, cy))

                # Prefer current target if safe
                if t.track_id == last_track_id and class1_boxes:
                    overlap = max([iou_box((x1, y1, x2, y2), b) for b in class1_boxes], default=0.0)
                    min_dist = min([center_distance((x1, y1, x2, y2), b) for b in class1_boxes], default=float('inf'))
                    if overlap > FORCE_DROP_IOU or min_dist < MIN_CENTER_DISTANCE or not is_path_safe(current_cursor_pos, (cx, cy), class1_boxes):
                        continue
                    best_track = t
                    best_distance = cursor_distance
                    break
                elif cursor_distance < best_distance:
                    if not class1_boxes or (
                        max([iou_box((x1, y1, x2, y2), b) for b in class1_boxes], default=0.0) <= SAFE_NEW_IOU and
                        min([center_distance((x1, y1, x2, y2), b) for b in class1_boxes], default=float('inf')) >= MIN_CENTER_DISTANCE and
                        is_path_safe(current_cursor_pos, (cx, cy), class1_boxes)
                    ):
                        best_track = t
                        best_distance = cursor_distance

            if best_track is not None:
                x1, y1, x2, y2 = best_track.to_ltrb()
                target_center = ((x1 + x2) / 2.0, (y1 + y2) / 2.0)
                chosen_track = best_track
                print(f"Selected track {best_track.track_id} with cursor distance: {best_distance:.2f}")

            # Aim prediction + move + click hold
            if auto_aim and target_center is not None:
                now = time.time()
                if hasattr(startAutoSlicer, "last_center") and startAutoSlicer.last_center is not None and startAutoSlicer.last_time:
                    dt = now - startAutoSlicer.last_time
                    if dt > 0:
                        vx = (target_center[0] - startAutoSlicer.last_center[0]) / dt
                        vy = (target_center[1] - startAutoSlicer.last_center[1]) / dt
                        latency = 0.08
                        pred_x = target_center[0] + vx * latency
                        pred_y = target_center[1] + vy * latency
                    else:
                        pred_x, pred_y = target_center
                else:
                    pred_x, pred_y = target_center

                if point_distance((pred_x, pred_y), current_cursor_pos) < MIN_MOVEMENT_THRESHOLD:
                    print(f"Skipping move to track {chosen_track.track_id}: movement too small")
                    path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                else:
                    startAutoSlicer.last_center = target_center
                    startAutoSlicer.last_time = now
                    mapped_x = int(MON["left"] + pred_x)
                    mapped_y = int(MON["top"] + pred_y)
                    mapped_x = max(0, min(SCREEN_W - 1, mapped_x))
                    mapped_y = max(0, min(SCREEN_H - 1, mapped_y))
                    target_lock = (mapped_x, mapped_y)

                    path_safe = not class1_boxes or is_path_safe(current_cursor_pos, (pred_x, pred_y), class1_boxes)
                    if chosen_track.track_id != last_track_id:
                        if mouse_pressed:
                            win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                            mouse_pressed = False
                            time.sleep(0.005)  # Reduced delay
                        if path_safe:
                            win32api.SetCursorPos(target_lock)
                            win32api.mouse_event(0x0002, 0, 0, 0, 0)  # Press
                            mouse_pressed = True
                            startAutoSlicer.last_track_id = chosen_track.track_id
                            path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                            print(f"Mouse moved to {target_lock}, clicked")
                        else:
                            print(f"Skipping move to track {chosen_track.track_id}: unsafe path")
                            target_lock = None
                            startAutoSlicer.last_track_id = None
                    else:
                        if path_safe:
                            win32api.SetCursorPos(target_lock)
                            if not mouse_pressed:
                                win32api.mouse_event(0x0002, 0, 0, 0, 0)  # Press
                                mouse_pressed = True
                            path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                            print(f"Mouse moved to {target_lock}, clicked (same track)")
                        else:
                            if mouse_pressed:
                                win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                                mouse_pressed = False
                            print(f"Releasing mouse: unsafe path for track {chosen_track.track_id}")
                            target_lock = None
                            startAutoSlicer.last_track_id = None
            else:
                if mouse_pressed:
                    win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                    mouse_pressed = False
                    time.sleep(0.005)
                target_lock = None
                startAutoSlicer.last_center = None
                startAutoSlicer.last_time = None
                startAutoSlicer.last_track_id = None
                print("Releasing mouse: no target or auto_aim off")

            elapsed = time.time() - t0
            fps = 1.0 / elapsed if elapsed > 0 else 0.0
            fps_deque.append(fps)
            print(f"Iteration FPS: {fps:.2f}")

            if visualize:
                frame = img.copy()
                cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(frame, f"Aim: {'ON' if auto_aim else 'OFF'}", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

                # Draw only selected track and class 1 boxes
                if chosen_track is not None:
                    x1, y1, x2, y2 = map(int, chosen_track.to_ltrb())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID {chosen_track.track_id} C0", (x1, max(0, y1-6)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                for x1, y1, x2, y2 in class1_boxes:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

                if target_center is not None:
                    cv2.circle(frame, (int(target_center[0]), int(target_center[1])), 4, (0, 255, 255), -1)
                if target_lock is not None:
                    vis_pred_x = target_lock[0] - MON["left"]
                    vis_pred_y = target_lock[1] - MON["top"]
                    cv2.circle(frame, (int(vis_pred_x), int(vis_pred_y)), 4, (255, 0, 255), -1)
                if path_to_draw is not None:
                    start, end = path_to_draw
                    cv2.line(frame, (int(start[0]), int(start[1])), (int(end[0]), int(end[1])),
                             (255, 165, 0), 1)

                cv2.imshow("screenshot", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    cv2.destroyAllWindows()
                    break

startAutoSlicer(visualize=True)